In [ ]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import re
import time
import pandas as pd
from selenium.common.exceptions import NoSuchElementException

# Chrome WebDriver 옵션 설정
options = webdriver.ChromeOptions()

# 웹 드라이버 경로 설정
webdriver_path = "/Users/dajungoh/Downloads/chromedriver_mac_arm64/chromedriver"

# ChromeDriver 서비스 생성
service = Service(webdriver_path)

# ChromeDriver를 사용하여 WebDriver 생성
driver = webdriver.Chrome(service=service)

# 웹페이지로 이동
url = "https://www.tripadvisor.com/Airline_Review-d8729134-Reviews-Qatar-Airways"
driver.get(url)

# 엑셀 파일 생성을 위한 데이터프레임 초기화
df = pd.DataFrame(columns=["비행 날짜", "별점", "비행 경로", "국제/국내", "이코노미/비즈니스", "리뷰 제목", "리뷰 텍스트"])

while True:
    # 페이지에서 Data 크롤링
    data = {}
    
    # 스크롤링을 통해 모든 리뷰 요소 로드
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # 대기 시간 설정
    
    rating_elements = driver.find_elements(By.CSS_SELECTOR, 'div.Hlmiy.F1[data-test-target="review-rating"] > span.ui_bubble_rating')

    routes = driver.find_elements(By.CSS_SELECTOR, 'div.tNvbV.f.Ng > div:nth-child(1)')
    internationals = driver.find_elements(By.CSS_SELECTOR, 'div.tNvbV.f.Ng > div:nth-child(2)')
    economy = driver.find_elements(By.CSS_SELECTOR, 'div.tNvbV.f.Ng > div:nth-child(3)')
    review_title = driver.find_elements(By.CSS_SELECTOR, 'div.KgQgP.MC._S.b.S6.H5._a')
    
    reviews = driver.find_elements(By.CSS_SELECTOR, 'div.vTVDc')
    date_elements = driver.find_elements(By.CSS_SELECTOR, 'span.teHYY._R.Me.S4.H3')    
    
    for i, review in enumerate(reviews):
        try:
            data["여행 날짜"] = date_elements[i].text.replace('Date of travel:', '').strip()

            read_more_link = review.find_element(By.CSS_SELECTOR, 'span.Ignyf._S.Z')
            if read_more_link.is_displayed():
                driver.execute_script("arguments[0].click();", read_more_link)
                time.sleep(0.5)  # 대기 시간 설정

            review_text_element = review.find_element(By.CSS_SELECTOR, 'div.fIrGe._T')
            review_text = review_text_element.text

            data['별점'] = int(rating_elements[i].get_attribute('class').split('_')[-1]) / 10
            data["경로"] = routes[i].text
            data["국제 여부"] = internationals[i].text
            data["이코노미 여부"] = economy[i].text
            data["리뷰 제목"] = review_title[i].text
            data["리뷰 텍스트"] = review_text
                        
            print(data)
            
            df = pd.concat([df, pd.DataFrame(data, index=[0])], ignore_index=True)

        except IndexError:
            break

    try:
        next_button = driver.find_element(By.XPATH, '//a[contains(@class, "ui_button nav next primary")]')
        if not next_button.is_enabled():
            break  # 다음 버튼이 비활성화되면 반복 종료
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(2)  # 대기 시간 설정
    except NoSuchElementException:
        break  # 다음 버튼이 없으면 반복 종료
        
# 엑셀 파일로 저장
df.to_excel("Qatar Airways.xlsx", index=False)

driver.quit()  